<a href="https://colab.research.google.com/github/holiz12/Analyse-Telco-Custer-Churn-using-XGBoost/blob/main/Telco_customer_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
url = 'https://raw.githubusercontent.com/holiz12/Analyse-Telco-Custer-Churn-using-XGBoost/main/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(url)

NameError: ignored